In [2]:
# !pip install ucimlrepo


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# 1. Download the data

In [14]:
from ucimlrepo import fetch_ucirepo 
  
cdc_diabetes_health_indicators = fetch_ucirepo(id=891) 
  
X = cdc_diabetes_health_indicators.data.features 
y = cdc_diabetes_health_indicators.data.targets 

print(cdc_diabetes_health_indicators.metadata) 


# Combine X and y to create a dataframe
import pandas as pd
df = pd.concat([X, y], axis=1)


df.head()

{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,1,1,1,40,1,0,0,0,0,1,...,0,5,18,15,1,0,9,4,3,0
1,0,0,0,25,1,0,0,1,0,0,...,1,3,0,0,0,0,7,6,1,0
2,1,1,1,28,0,0,0,0,1,0,...,1,5,30,30,1,0,9,4,8,0
3,1,0,1,27,0,0,0,1,1,1,...,0,2,0,0,0,0,11,3,6,0
4,1,1,1,24,0,0,0,1,1,1,...,0,2,3,0,0,0,11,5,4,0


In [19]:
# Filter any person who has not had their cholesterol checked in the last 5 years
df = df[df['CholCheck'] == 1]
df = df.drop(columns=['CholCheck'])

In [20]:
# Combine Fruits and Veggies into a single Healthy Eating Score
df['HealthyEating'] = df['Fruits'] + df['Veggies']
df = df.drop(columns=['Fruits', 'Veggies'])

In [22]:
df.head()

,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary,HealthyEating
0,1,1,40,1,0,0,0,0,1,0,5,18,15,1,0,9,4,3,0,1
2,1,1,28,0,0,0,0,0,1,1,5,30,30,1,0,9,4,8,0,1
3,1,0,27,0,0,0,1,0,1,0,2,0,0,0,0,11,3,6,0,2
4,1,1,24,0,0,0,1,0,1,0,2,3,0,0,0,11,5,4,0,2
5,1,1,25,1,0,0,1,0,1,0,2,0,2,0,1,10,6,8,0,2


In [23]:
df.to_csv('cdc_diabetes_health_indicators.csv', index=False)